In [9]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('horse_data.csv', names = ['Fact of surgery', 'Age', 'Hospital Number', 'Temperature', 'Pulse', 'Respiratory Rate', 'Temperature of Extremities', 'Peripheral Pulse', 'Mucous Membranes', 'Capillary Refill Time','Pain', 'Peristalsis', 'Abdominal Distension', 'Nasogastric Tube', 'Nasogastric Reflux', 'Nasogastric Reflux PH', 'Rectal Examination', 'Abdomen', 'Packed Cell Volume', 'Total Protein', 'Abdominocentesis Appearance', 'Abdomcentesis Total Protein', 'Outcome', 'Surgical Lesion', 'Type of Lesion', 'Type of Lesion_1', 'Type of Lesion_2', 'cp_data'], sep = '\,', header=0, engine='python')
df.head(5)

In [ ]:
#Колонки для анализа

data = df[['Hospital Number', 'Fact of surgery', 'Temperature', 'Pulse', 'Abdominal Distension', 'Packed Cell Volume','Pain', 'Outcome']].reset_index()
data.info()

In [ ]:
data = data.replace('?', np.nan)
data = data.apply(pd.to_numeric)
data.info()

In [ ]:
data.describe()

In [ ]:
#выбросы по температуре

def ejection(row):
    q1 = data['Temperature'].quantile(0.25)
    q3 = data['Temperature'].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr) 
    upper_bound = q3 + (1.5 * iqr)

    if row['Temperature'] > upper_bound or row['Temperature'] < lower_bound:
        return row['Temperature']

In [ ]:
#Возможный разброс температуры для лошадей от 35 до 42 град Цельсия, где 40 и выше соответствует лихорадке, а 36 и ниже - шоковому состоянию. Все выбросы находятся в рамках допустимых значений.
data_1 = pd.DataFrame()
data_1['Temperature ejection'] = data.apply(ejection, axis=1)
data_1.dropna()

In [ ]:
#Выбросы по пульсу

def ejection_1(row):
    q1 = data['Pulse'].quantile(0.25)
    q3 = data['Pulse'].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr) 
    upper_bound = q3 + (1.5 * iqr)

    if row['Pulse'] > upper_bound or row['Pulse'] < lower_bound:
        return row['Pulse']

In [ ]:
data_1 = pd.DataFrame()
data_1['Pulse ejection'] = data.apply(ejection_1, axis=1)
data_1.dropna()

#Физиологическая норма для лошадей от 20 до 240 ударов. Все выбросы имеют возможные значения

In [ ]:
def ejection_3(row):
    q1 = data['Packed Cell Volume'].quantile(0.25)
    q3 = data['Packed Cell Volume'].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr) 
    upper_bound = q3 + (1.5 * iqr)

    if row['Packed Cell Volume'] > upper_bound or row['Packed Cell Volume'] < lower_bound:
        return row['Packed Cell Volume']


In [ ]:
data_1 = pd.DataFrame()
data_1['PCV ejection'] = data.apply(ejection_3, axis=1)
data_1.dropna()
#максимальный гематокрит - 80%, повышенные значения могут быть вызваны обезвоживанием организма, осложнением после операции и тд

In [ ]:
for col in data:
    pct_missing = data[col].isnull().mean()
    print(f'{col} - {pct_missing :.1%}')

In [ ]:
#найдем пропуск по факту операции
data[data['Fact of surgery'].isnull()]

In [ ]:
# так как значения температуры, пульсы и гематокрита находятся в рамках показаний для здоровой лошади, вздутие отсутствовало, предположим, что показаний к операции не было. Внесем значение вручную
data['Fact of surgery'].fillna(2, inplace=True)

In [ ]:
#пропущенное значение исхода заполним исходя из группировок по боли и операции - чем выше эти показания, тем в среднем хуже исход

data.loc[data['Outcome'].isnull(), 'Outcome'] = data.groupby(['Fact of surgery','Pain'])['Outcome'].transform('median')

In [ ]:
#из описания данных можно сделать вывод, что чем сильнее вздутие, тем вероятнее операция. Заполним пропуски по столбцу вздутия на основе проведенных операций
data.loc[data['Abdominal Distension'].isnull(), 'Abdominal Distension'] = data.groupby(['Fact of surgery'])['Abdominal Distension'].transform('median')

In [ ]:
#пропуски в графе боль заполняем на основании вздутия. Чем оно больше, тем больше боли должно испытывать животное
data.loc[data['Pain'].isnull(), 'Pain'] = data.groupby(['Abdominal Distension'])['Pain'].transform('median')

In [ ]:
#показатели температуры, пульса и гематокрита заполним медианами соответствующих значений
data['Temperature'].fillna(data['Temperature'].median(), inplace=True)

In [ ]:
data['Pulse'].fillna(data['Pulse'].median(), inplace=True)

In [ ]:
data['Packed Cell Volume'].fillna(data['Packed Cell Volume'].median(), inplace=True)

In [ ]:
data.info()